# 🌈 1. What is Ordinal Encoding?
##### 🧠 Concept:

##### **Ordinal Encoding is a technique to convert categorical (textual) features into numeric values, while preserving the order (ranking)** among the categories.

##### Example:

| Size   | Meaning |
| ------ | ------- |
| Small  | Lowest  |
| Medium | Middle  |
| Large  | Highest |

##### If you encode:

```mathematica
    Small → 0  
    Medium → 1  
    Large → 2

```
##### Here, the **numerical values reflect an order or ranking between categories — this is why it’s called Ordinal Encoding.**

<HR>


## 🚦 2. When to Use Ordinal Encoding

##### Use it **when the categories have a natural order**.

##### ✅ Examples of **Ordered Categories**

| Feature      | Categories                         | Ordinal Encoding Example |
| ------------ | ---------------------------------- | ------------------------ |
| Size         | Small, Medium, Large               | 0, 1, 2                  |
| Education    | High School, Bachelor, Master, PhD | 0, 1, 2, 3               |
| Satisfaction | Poor, Average, Good, Excellent     | 0, 1, 2, 3               |


#### ❌ Do NOT use it when:
- There’s **no order** (like “Red”, “Green”, “Blue” or “Male”, “Female”).
- In those cases, use **Label Encoding** (if ordinal meaning doesn’t matter) **or One-Hot Encoding.**
  

<HR>


## ⚙️ 3. How Ordinal Encoding Works (Behind the Scenes)

##### It maps each category to an integer.
##### The mapping can be:

- **Automatic** (default alphabetical order)

- **Custom-defined** (you define the order manually)

For example:

#### **Automatic mode:** (Default: alphabetical order)

```python

    ['Large', 'Medium', 'Small'] →  [0, 1, 2]

```
#### **Custom order (preferred for ordinal features):** (This preserves the true order)

```python
    ['Small', 'Medium', 'Large'] →  [0, 1, 2]
```
<HR>

## 🧩 4. Ordinal Encoding in Python

In [49]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder

#### ➡️ Creating the dataset

In [50]:
df = pd.DataFrame({"Size": ["s", "m", "l", "xl", "s", "m", "l", "s", "s", "l", "xl", "m"]})
df.head()

,Size
0,s
1,m
2,l
3,xl
4,s


#### ➡️ Applying Ordinal Encoding (Automatic)

In [51]:
# created an OrdinalEncoder Object
encoder = OrdinalEncoder()

#Fit and transform the data
encoded_data = encoder.fit_transform(df)

encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out())

encoded_df.head() # as we can see the encoding is in Alphabetical Order which does not adhere to real world logic

,Size
0,2.0
1,1.0
2,0.0
3,3.0
4,2.0


#### 🐴 **Hold your horses** : **Applying Custom Order**

##### ➡️ Defining Category order

In [52]:
# ordered_data = [["s","m","l","xl"]] # this also works
ordered_data = ["s","m","l","xl"] 

##### ➡️ Initiating Ordinal Encoder with Categories

In [53]:
oe = OrdinalEncoder(categories= [ordered_data] ) 
#categories parameter takes the argument of list of different categories order

In [54]:
oe.fit(df[["Size"]])

OrdinalEncoder(categories=[['s', 'm', 'l', 'xl']])

In [55]:
df["Size_en"] = oe.transform(df[["Size"]])

In [56]:
df

,Size,Size_en
0,s,0.0
1,m,1.0
2,l,2.0
3,xl,3.0
4,s,0.0
5,m,1.0
6,l,2.0
7,s,0.0
8,s,0.0
9,l,2.0


#### 💡 Understanding What `.fit_transform()` Does Here

Let’s break it down:

1. `.fit()` → Learns which categories exist in each feature and stores the mapping.

2. `.transform()` → Converts the original categories into their corresponding numbers based on that mapping.

3. `.fit_transform()` = Does both in one step.

In [57]:
# Inspect Encoded Mappings
print(oe.categories_)

[array(['s', 'm', 'l', 'xl'], dtype=object)]


#### 🧾 Decode Back (Inverse Transform)

In [58]:
df["Decoded_data"] = oe.inverse_transform(df[['Size_en']]).flatten()
df

,Size,Size_en,Decoded_data
0,s,0.0,s
1,m,1.0,m
2,l,2.0,l
3,xl,3.0,xl
4,s,0.0,s
5,m,1.0,m
6,l,2.0,l
7,s,0.0,s
8,s,0.0,s
9,l,2.0,l


### 🌟 Important NOTE:

- #### When you assign a value to a new column in a pandas DataFrame `(df["Decoded_data"] = value)`, pandas expects value to be a 1-dimensional array (a pandas Series or a NumPy array with shape `(n_samples,)`). 
- #### A 2D array cannot be directly assigned to a single column. The **`ValueError: 2 often indicates that the object being assigned has ndim=2`.**
  
### ➡️ 🌟 🌟 🌟 This Pandas Lib 🐼 is a Nigga : it is smart enough to `.flatten()` the **2-D numpy array** returned by **`.fit_transform`** if it has numerical data But in case of the **Categorical Data** it becomes **Dumb as a sloth** it is not able to infer / flatten the 2D numpy array of categorical data returned by **`.inverse_transform()`** thats why before adding it to the new column in the dataFrame we either have to flatten it or make it into a pd.series or dataframe

### ⚠️ Common Mistakes

| Mistake                                        | Why it’s a Problem                                                       |
| ---------------------------------------------- | ------------------------------------------------------------------------ |
| Using Ordinal Encoding on unordered categories | Creates fake relationships (e.g. Male=0, Female=1 implies Female > Male) |
| Forgetting to define custom order              | Defaults to alphabetical order (which may be meaningless)                |
| Encoding training and test data separately     | Category mapping may mismatch between them                               |


**👉 Always use the same encoder instance (fit on train, transform on test).**



### 📊 Comparison with Other Encoders

| Encoder Type    | Preserves Order? | Suitable for          | Output Example                  |
| --------------- | ---------------- | --------------------- | ------------------------------- |
| Label Encoder   | ❌ No             | Target/Single Feature | `Red → 0, Blue → 1`             |
| Ordinal Encoder | ✅ Yes            | Ordered Features      | `Low → 0, Medium → 1, High → 2` |
| One-Hot Encoder | ❌ No             | Unordered Features    | `[1,0,0], [0,1,0], [0,0,1]`     |


In [59]:
#  one more example from loan.csv
dataset = pd.read_csv("loan_data_set.csv")

dataset.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [60]:
dataset["Property_Area"].unique()

array(['Urban', 'Rural', 'Semiurban'], dtype=object)

In [61]:
dataset["Property_Area"].fillna(dataset["Property_Area"].mode()[0],inplace = True)

/tmp/ipykernel_9147/2105752046.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dataset["Property_Area"].fillna(dataset["Property_Area"].mode()[0],inplace = True)


In [62]:
ord_data = ["Rural","Semiurban","Urban"]

In [63]:
oe1 = OrdinalEncoder(categories=[ord_data])

In [64]:
dataset["Property_Area"] = oe1.fit_transform(dataset[["Property_Area"]])

In [65]:
dataset["Property_Area"].unique()

array([2., 0., 1.])